In [1]:
import numpy as np
import pandas as pd

import pickle

from pitched_recommend import Recommender

In [2]:
class MacOSFile(object):
    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        if n >= (1 << 31):
            buffer = bytearray(n)
            pos = 0
            while pos < n:
                size = min(n - pos, 1 << 31 - 1)
                chunk = self.f.read(size)
                buffer[pos:pos + size] = chunk
                pos += size
            return buffer
        return self.f.read(n)

In [3]:
with open("/Users/daria/Downloads/prod_saved_model (1).pkl", "rb") as fin:
    model = pickle.load(MacOSFile(fin))

In [4]:
with open('/Users/daria/Downloads/prod_metadata_dict.pkl', 'rb') as pickle_file:
    metadata = pickle.load(pickle_file)

In [31]:
songpickr2019 = pd.read_csv('/Users/daria/Downloads/https___pitched-optimize.umusic.net_exports_holgerchristoph_20190429142336.csv')
songpickr2019.head()

,PlaylistUri,Name,Description,Image,Isrc,Artist,TrackName,Label,Duration,AddedAt,Upc
0,spotify:playlist:6YwIqXS0auLevGXID24D7C,"Songpickr: 2019 Best Songs (Indie Rock, Folk, ...","Warm, organic, authentic, timeless songs curat...",https://pl.scdn.co/images/pl/default/f199c4493...,USATO1900023,Black Pumas,Colors,"2019 ATO Records, under exclusive license to [...",246586,2019-04-17T16:56:26Z,5.400863e+12
1,spotify:playlist:6YwIqXS0auLevGXID24D7C,"Songpickr: 2019 Best Songs (Indie Rock, Folk, ...","Warm, organic, authentic, timeless songs curat...",https://pl.scdn.co/images/pl/default/f199c4493...,USAT21901822,Wilder Woods,Sure Ain't,2019 Atlantic Recording Corporation,190960,2019-04-13T10:48:56Z,7.567985e+10
2,spotify:playlist:6YwIqXS0auLevGXID24D7C,"Songpickr: 2019 Best Songs (Indie Rock, Folk, ...","Warm, organic, authentic, timeless songs curat...",https://pl.scdn.co/images/pl/default/f199c4493...,USATO1900004,Dylan LeBlanc,Born Again,"2019 ATO Records, under exclusive license to [...",210921,2019-04-24T15:21:42Z,5.400863e+12
3,spotify:playlist:6YwIqXS0auLevGXID24D7C,"Songpickr: 2019 Best Songs (Indie Rock, Folk, ...","Warm, organic, authentic, timeless songs curat...",https://pl.scdn.co/images/pl/default/f199c4493...,USC4R1906800,Tyler Ramsey,Breaking A Heart,"© 2019 Fantasy Records, a division of Concord ...",293946,2019-04-13T14:34:04Z,8.880721e+11
4,spotify:playlist:6YwIqXS0auLevGXID24D7C,"Songpickr: 2019 Best Songs (Indie Rock, Folk, ...","Warm, organic, authentic, timeless songs curat...",https://pl.scdn.co/images/pl/default/f199c4493...,USUM71810629,J.S. Ondara,American Dream,"© 2018 Verve Label Group, a Division of UMG Re...",240200,2019-04-22T18:58:28Z,6.025679e+11


In [32]:
in_model = songpickr2019[songpickr2019.Isrc.isin(model.isrcs)]

In [33]:
songpickr2019.shape

(100, 11)

In [34]:
in_model.shape

(62, 11)

In [35]:
in_model = in_model.drop(columns=['Name','Description','Image','Label','Duration','Upc','AddedAt'])

col_dict = {
    'PlaylistUri':'playlist_uri',
    'Isrc':'isrc',
    'Artist':'track_artist',
    'TrackName':'track_title',
    'AddedAt':'date_add'
}

in_model = in_model.rename(columns=col_dict)
in_model.head()

,playlist_uri,isrc,track_artist,track_title
4,spotify:playlist:6YwIqXS0auLevGXID24D7C,USUM71810629,J.S. Ondara,American Dream
5,spotify:playlist:6YwIqXS0auLevGXID24D7C,USC4R1802568,Andrew Bird,Sisyphus
6,spotify:playlist:6YwIqXS0auLevGXID24D7C,USWB11801818,Dee White,Wherever You Go
7,spotify:playlist:6YwIqXS0auLevGXID24D7C,USAT21811675,Rival Sons,Feral Roots
8,spotify:playlist:6YwIqXS0auLevGXID24D7C,US27Q1842414,The Nude Party,Chevrolet Van


In [36]:
in_model['days_on_playlist'] = 30

In [37]:
in_model = in_model.reset_index(drop=True)

In [38]:
songpickr2019_recs = model.recommend_outofmodel(in_model,model.playlists_sparse, N=300)

/usr/local/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


In [39]:
songpickr2019_recs_readable = model.rec_to_isrc(songpickr2019_recs)

In [40]:
songpickr2019_recs_final = [(x[0],x[1],metadata['track_artist'][x[0]], metadata['track_title'][x[0]], metadata['major_label'][x[0]], metadata['label_studio'][x[0]],
               metadata['original_release_date'][x[0]], metadata['genre_name'][x[0]], metadata['parent_genre_name'][x[0]], 
               x[2]) for x in list(songpickr2019_recs_readable)]

all_columns = ['isrc','score','artist','title','major_label','label_studio','original_release_date','genre_name',
               'parent_genre_name','seed_playlist_uri']
songpickr2019_recs_final_df = pd.DataFrame(songpickr2019_recs_final, columns = all_columns)

In [41]:
songpickr2019_recs_final_df

,isrc,score,artist,title,major_label,label_studio,original_release_date,genre_name,parent_genre_name,seed_playlist_uri
0,USJ5G1817101,1.259827,Strand of Oaks,Weird Ways,Other,Dead Oceans,2019-03-22,Indie Rock,Alternative,spotify:playlist:6YwIqXS0auLevGXID24D7C
1,USUM71810630,1.256112,J.S. Ondara,Saying Goodbye,UMG,Verve Forecast,2019-02-01,Singer/Songwriter,Singer/Songwriter,spotify:playlist:6YwIqXS0auLevGXID24D7C
2,QZ6SA1800090,1.230105,Yola,Faraway Look,Warner,Easy Eye Sound/Nonesuch,2019-02-22,Rock,Rock,spotify:playlist:6YwIqXS0auLevGXID24D7C
3,USYAH1700389,1.207086,Jade Bird,I Get No Joy,Other,Glassnote Music LLC,2019-04-19,Alternative,Alternative,spotify:playlist:6YwIqXS0auLevGXID24D7C
4,USUG11900041,1.195176,Ryan Adams,Fuck The Rain,UMG,Blue Note Records,2019-01-23,Rock,Rock,spotify:playlist:6YwIqXS0auLevGXID24D7C
5,USD8D1960006,1.184915,Nick Waterhouse,Wreck The Rod,Other,"Innovative Leisure, distributed by Inertia Music",2019-03-08,Rock,Rock,spotify:playlist:6YwIqXS0auLevGXID24D7C
6,USJ5G1918803,1.157972,Better Oblivion Community Center,Dylan Thomas,Other,Dead Oceans,2019-01-23,Indie Rock,Alternative,spotify:playlist:6YwIqXS0auLevGXID24D7C
7,USWB11803412,1.144205,Jenny Lewis,Red Bull & Hennessy,Warner,Warner Bros.,2019-03-22,Alternative,Alternative,spotify:playlist:6YwIqXS0auLevGXID24D7C
8,USBQU1800148,1.138360,Molly Sarle,Human,Other,Partisan Records,2019-01-28,Alternative,Alternative,spotify:playlist:6YwIqXS0auLevGXID24D7C
9,USAT21811359,1.123818,Adia Victoria,Different Kind Of Love,Unknown,Unknown,Unknown,Unknown,Unknown,spotify:playlist:6YwIqXS0auLevGXID24D7C


In [42]:
songpickr2019_recs_final_df.to_csv('/Users/daria/Downloads/songpickr2019_test_recommendations.csv')